In [ ]:
! pip -q install transformers wandb pytorch-lightning

In [ ]:
import os
import logging
import pickle
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import pytorch_lightning as pl
from transformers import AutoTokenizer, AutoModelForCausalLM, get_linear_schedule_with_warmup
import wandb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torchmetrics

In [ ]:
SEED = 42
def set_seed(seed: int = 42, set_torch=True):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    if set_torch:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_seed(SEED)

# Test Chat

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-small', padding_side='left')
# model = AutoModelForCausalLM.from_pretrained('tinkoff-ai/ruDialoGPT-small')

In [ ]:
# chat_history_ids = ""

# for step in range(5):
#     chat_history_ids = chat_history_ids + "@@ПЕРВЫЙ@@ " + input(">> User: ") + "@@ВТОРОЙ@@"
#     new_input_ids = tokenizer(chat_history_ids, return_tensors='pt')
#     generated_token_ids = model.generate(
#         **new_input_ids,
#         top_k=10,
#         top_p=0.95,
#         num_beams=3,
#         num_return_sequences=1,
#         do_sample=True,
#         no_repeat_ngram_size=2,
#         temperature=1.7,
#         repetition_penalty=1.2,
#         length_penalty=1.0,
#         eos_token_id=50257,
#         max_new_tokens=40,
#         pad_token_id=tokenizer.eos_token_id
#     )

#     context_with_response = tokenizer.decode(generated_token_ids[0])
#     cutted_answer = context_with_response[len(chat_history_ids):]
#     if "@@ПЕРВЫЙ@@" in cutted_answer:
#         cutted_answer = cutted_answer.split("@@ПЕРВЫЙ@@")[0]
#     if "@@ВТОРОЙ@@" in cutted_answer:
#         cutted_answer = cutted_answer.split("@@ВТОРОЙ@@")[0]
#     chat_history_ids = chat_history_ids + cutted_answer
#     print(f"ruDialoGPT: ", cutted_answer)

# Args

In [ ]:
class Config():
    def __init__(self):
        self.seed = 42
        self.word_dropout = 0
        self.batch_size = 2
        self.val_size = 0.2
        self.learning_rate = 1e-4
        # self.txt_in_min_len = 2
        # self.txt_in_max_len = 8
        # self.txt_out_min_len = 4
        # self.txt_out_max_len = 16
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.max_epochs = 5
        self.total_steps = int(13832 * self.max_epochs)
        self.warmup_steps = int(0.05 * self.total_steps)

cfg = Config()

# Data

In [ ]:
data = pd.read_csv("/content/data2.csv")
data.sample(5)

,context_3,context_2,context_1,response
5828,NaN,NaN,NaN,в clearml можно тегами версионировать и фильтр...
6729,NaN,NaN,NaN,А зеркально или параллельно учить на русском и...
12126,NaN,NaN,"Очень легко сделать симуляцию в GTA5, использу...",Есть какой-нибудь гайд как это сделать? Уже да...
2974,NaN,NaN,NaN,"Хм, осталось 4 реакции"
2308,NaN,NaN,"привет всем! \n\nя Сюзанна, \nr&d в RobotMIA, ...",А что делаете в RobotMIA если не секрет?


In [ ]:
data.shape

(13832, 4)

In [ ]:
# data_clear = data.dropna()

In [ ]:
data_filled = data.fillna("")
data_filled.sample(3)

,context_3,context_2,context_1,response
7164,,,,Да одна нозологическая группа\nИнтересно
7553,,может смотреть на количество слогов в строчке?...,может не рифмоваться же,там же разные варианты есть и они конечны. каж...
573,,"Чат для нейронок в области биологии, химии, фи...",Ага уже давно белки и хим связи с помощью Grap...,В молекулах генерацию на графах не особо делаю...


# Model

In [ ]:
class ApplyWordDropout:
    def __init__(self, replace_with, eos_token_id, word_dropout=0.0):
        self.keep_prop = 1.0 - word_dropout
        self.replace_with = replace_with
        self.eos_token_id = eos_token_id

    def _apply_word_dropout(self, tensor):
        dropout_mask = torch.rand(tensor.shape) < self.keep_prop
        dropout_mask &= tensor != self.eos_token_id
        result = torch.where(dropout_mask, tensor, torch.full_like(tensor, self.replace_with))
        return result

    def __call__(self, sample):
        return self._apply_word_dropout(sample)

In [ ]:
class ConversationDataset(Dataset):
    def __init__(self, df, cfg):
        self.tokenizer = AutoTokenizer.from_pretrained(
            'tinkoff-ai/ruDialoGPT-small',
            padding_side='left'
        )
        self.word_dropout = ApplyWordDropout(
            replace_with=self.tokenizer(self.tokenizer.unk_token)['input_ids'][0],
            eos_token_id=self.tokenizer.eos_token_id,
            word_dropout=cfg.word_dropout,
        )
        self.samples = []
        for _, sentences in df.iterrows():
            conv = self._concat_conv(sentences, self.tokenizer)
            self.samples.append(conv)
        if cfg.word_dropout:
            self.samples = [self.word_dropout(sample) for sample in self.samples]

    def _concat_conv(self, sentences, tokenizer):
        eos_list = [50257, 50258, 50257, 50258, 50257]
        conv = [
            torch.cat(
                (
                    torch.tensor([eos_list.pop()]).unsqueeze(0),
                    tokenizer(sentence, return_tensors="pt")["input_ids"],
                ),
                dim=1,
            )
            for sentence in sentences
            if sentence != ""
        ]
        conv[-1] = torch.cat(
            (
                conv[-1],
                torch.tensor([eos_list.pop()]).unsqueeze(0),
            ),
            dim=1,
        )
        conv_flat = torch.cat(conv, dim=1).view(-1)
        return conv_flat

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, item):
        return self.samples[item].to(torch.long)

In [ ]:
class ConversationDataModule(pl.LightningDataModule):
    def __init__(self, data, cfg):
        super().__init__()
        train_data, val_data = train_test_split(data, test_size=cfg.val_size)
        self.train_data = train_data
        self.val_data = val_data
        self.cfg = cfg

    def setup(self, stage=None):
        self.train_dataset = ConversationDataset(self.train_data, self.cfg)
        self.val_dataset = ConversationDataset(self.val_data, self.cfg)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.cfg.batch_size, shuffle=True, collate_fn=self._collate)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.cfg.batch_size, collate_fn=self._collate)

    # def _collate(self, examples: list[torch.Tensor]):
    #     return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)
    def _collate(self, examples: list[torch.Tensor]):
        max_length = max([len(ex) for ex in examples])
        padded_examples = [F.pad(ex, (max_length - len(ex), 0)) for ex in examples]
        return torch.stack(padded_examples, dim=0)

In [ ]:
class DialoTuner(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.model = AutoModelForCausalLM.from_pretrained("tinkoff-ai/ruDialoGPT-small")
        self.cfg = cfg
        self.perplexity = torchmetrics.text.Perplexity()

    def forward(self, batch):
        inputs, labels = (batch, batch)
        return self.model(input, labels=labels)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.cfg.learning_rate)
        # Calculate the total number of training steps
        total_steps = self.cfg.total_steps
        # Create the scheduler with linear warmup and decay
        lr_scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.cfg.warmup_steps,
            num_training_steps=total_steps
        )

        return [optimizer], [lr_scheduler]

    def training_step(self, train_batch, batch_idx):
        inputs, labels = (train_batch, train_batch)
        outputs = self.model(inputs, labels=labels)
        loss = outputs[0]
        self.log("train_loss", loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        inputs, labels = (val_batch, val_batch)
        outputs = self.model(inputs, labels=labels)
        loss = outputs[0]
        perplexity_score = self.perplexity(outputs.logits, labels)
        self.log("val_loss", loss)
        self.log("val_perplexity", perplexity_score)
        return loss

    def generate(self, **kwargs):
        return self.model.generate(**kwargs)

# Trainer

In [ ]:
# conversation_data_module = ConversationDataModule(data_filled, cfg)
# model = DialoTuner(cfg)

In [ ]:
# import wandb

# key = "5d9fffc78b7b93676008c4a6a8ac3b28a11dd421"
# wandb.login(key=key)

# config_dict = {attr: getattr(cfg, attr) for attr in dir(cfg) if not callable(getattr(cfg, attr)) and not attr.startswith("__")}
# logger = pl.loggers.WandbLogger(project='sandbox', config=config_dict, log_model=True)

In [ ]:
# trainer = pl.Trainer(
#     accelerator="gpu",
#     devices=[0],
#     max_epochs=cfg.max_epochs,
#     logger=logger,
#     log_every_n_steps=1,
#     gradient_clip_val=1.0,
# )

In [ ]:
# trainer.fit(model, datamodule=conversation_data_module)

In [ ]:
# trainer.validate(model, datamodule=conversation_data_module)

In [ ]:
# torch.save(model, 'finetuned_model_10ep_1e4.pth')

In [ ]:
# wandb.finish()

# Simple Interface

In [ ]:
# chat_history_ids = ""
# tokenizer = AutoTokenizer.from_pretrained(
#             'tinkoff-ai/ruDialoGPT-small',
#             padding_side='left'
#         )


# for step in range(5):
#     chat_history_ids = chat_history_ids + "@@ПЕРВЫЙ@@ " + input(">> User: ") + "@@ВТОРОЙ@@"
#     new_input_ids = tokenizer(chat_history_ids, return_tensors='pt')
#     generated_token_ids = model.generate(
#         **new_input_ids,
#         top_k=10,
#         top_p=0.95,
#         num_beams=3,
#         num_return_sequences=1,
#         do_sample=True,
#         no_repeat_ngram_size=2,
#         temperature=1.7,
#         repetition_penalty=1.2,
#         length_penalty=1.0,
#         eos_token_id=50257,
#         max_new_tokens=40,
#         pad_token_id=tokenizer.eos_token_id
#     )

#     context_with_response = tokenizer.decode(generated_token_ids[0])
#     cutted_answer = context_with_response[len(chat_history_ids):]
#     if "@@ПЕРВЫЙ@@" in cutted_answer:
#         cutted_answer = cutted_answer.split("@@ПЕРВЫЙ@@")[0]
#     if "@@ВТОРОЙ@@" in cutted_answer:
#         cutted_answer = cutted_answer.split("@@ВТОРОЙ@@")[0]
#     chat_history_ids = chat_history_ids + cutted_answer
#     print(f"ruDialoGPT: ", cutted_answer)

# LoRA

In [ ]:
!pip install "peft" --quiet
!pip install "accelerate" "evaluate" loralib --upgrade --quiet
!pip install accelerate -U --quiet

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-small', padding_side='left')
model = AutoModelForCausalLM.from_pretrained('tinkoff-ai/ruDialoGPT-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
#  target_modules=["q", "v"],
    target_modules=["wte", "lm_head"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

# model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model.print_trainable_parameters()

trainable params: 3,265,856 || all params: 128,495,168 || trainable%: 2.5416177517274425


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Since you're doing regular language modeling, set mlm=False
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    optim="adamw_torch",
    output_dir="./output",
    report_to ="wandb",
    evaluation_strategy='steps',
    eval_steps=3000,
    overwrite_output_dir=True,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    save_steps=500,
    logging_steps=100,
    fp16=True,
    prediction_loss_only=True
)

In [ ]:
%env WANDB_LOG_MODEL=true

key = "5d9fffc78b7b93676008c4a6a8ac3b28a11dd421"
wandb.login(key=key)

env: WANDB_LOG_MODEL=true


wandb: Currently logged in as: whiteroseraf (hackathon-fu). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
perplexity_metric = torchmetrics.text.Perplexity()

def compute_perplexity(eval_pred):
    predictions, labels = eval_pred
    perplexity = perplexity_metric(predictions, labels)
    return {"perplexity": perplexity}

In [ ]:
from transformers import Trainer

conversation_data_module = ConversationDataModule(data_filled, cfg)
conversation_data_module.setup()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = model.to(cfg.device)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=conversation_data_module.train_dataset,
    eval_dataset=conversation_data_module.val_dataset,
    # compute_metrics=compute_perplexity,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
3000,4.544000,4.579616
6000,4.370800,4.465733
9000,4.297500,4.414749
12000,4.241900,4.391315


TrainOutput(global_step=13830, training_loss=4.443358155289855, metrics={'train_runtime': 1701.8915, 'train_samples_per_second': 32.508, 'train_steps_per_second': 8.126, 'total_flos': 2889397055857536.0, 'train_loss': 4.443358155289855, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 4.387857437133789,
 'eval_runtime': 59.1005,
 'eval_samples_per_second': 46.819,
 'eval_steps_per_second': 46.819,
 'epoch': 5.0}

In [ ]:
wandb.finish()

eval/loss,█▄▂▁▁
eval/runtime,▆█▁██
eval/samples_per_second,▃▁█▁▁
eval/steps_per_second,▃▁█▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▆▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
path="lora-1ep"

In [ ]:
trainer.save_model(path)

In [ ]:
model.save_pretrained(path + "pretrained")

In [ ]:
# model = model.cpu()

In [ ]:
# chat_history_ids = ""

# for step in range(5):
#     chat_history_ids = chat_history_ids + "@@ПЕРВЫЙ@@ " + input(">> User: ") + "@@ВТОРОЙ@@"
#     new_input_ids = tokenizer(chat_history_ids, return_tensors='pt')
#     generated_token_ids = model.generate(
#         **new_input_ids,
#         top_k=10,
#         top_p=0.95,
#         num_beams=3,
#         num_return_sequences=1,
#         do_sample=True,
#         no_repeat_ngram_size=2,
#         temperature=1.7,
#         repetition_penalty=1.2,
#         length_penalty=1.0,
#         eos_token_id=50257,
#         max_new_tokens=40,
#         pad_token_id=tokenizer.eos_token_id
#     )

#     context_with_response = tokenizer.decode(generated_token_ids[0])
#     cutted_answer = context_with_response[len(chat_history_ids):]
#     if "@@ПЕРВЫЙ@@" in cutted_answer:
#         cutted_answer = cutted_answer.split("@@ПЕРВЫЙ@@")[0]
#     if "@@ВТОРОЙ@@" in cutted_answer:
#         cutted_answer = cutted_answer.split("@@ВТОРОЙ@@")[0]
#     chat_history_ids = chat_history_ids + cutted_answer
#     print(f"ruDialoGPT: ", cutted_answer)

>> User: на работу кто возьмет?
ruDialoGPT:  Я бы взял, но у меня нет работы ((
>> User: а данные есть?
ruDialoGPT:  Есть конечно, просто я работаю на работе, и мне нужно их куда-нибудь девать. Но это не точно, я не уверен, что смогу это сделать в ближайшее время. 😁
>> User: у гпт спроси
ruDialoGPT:  У них есть датасеты в инете, можно посмотреть, если надо, то могу скинуть. Я не знаю, как их можно использовать в личных целях) Спасибо за инфу) Буду рад


KeyboardInterrupt: ignored

In [ ]:
input_str = "где веса ламы найти?"

chat_history_ids = "@@ПЕРВЫЙ@@ " + input_str + "@@ВТОРОЙ@@"
new_input_ids = tokenizer(chat_history_ids, return_tensors='pt')
generated_token_ids = model.generate(
        **new_input_ids,
        top_k=10,
        top_p=0.95,
        num_beams=3,
        num_return_sequences=3,
        do_sample=True,
        no_repeat_ngram_size=2,
        temperature=1.4,
        repetition_penalty=1.2,
        length_penalty=1.0,
        eos_token_id=50257,
        max_new_tokens=40,
        pad_token_id=tokenizer.eos_token_id
)

context_with_response = [tokenizer.decode(generated_token_ids[i]) for i in range(len(generated_token_ids))]
context_with_response

['@@ПЕРВЫЙ@@ где веса ламы найти?@@ВТОРОЙ@@в гугле вбивай датасеты и там найдёшь весы с ламой@@ПЕРВЫЙ@@<|endoftext|>',
 '@@ПЕРВЫЙ@@ где веса ламы найти?@@ВТОРОЙ@@в гугле вбивай датасеты и там найдёшь весы с ламами@@ПЕРВЫЙ@@<|endoftext|>',
 '@@ПЕРВЫЙ@@ где веса ламы найти?@@ВТОРОЙ@@в гугле вбивай датасеты и там найдёшь весы с ламой.@@ПЕРВЫЙ@@']